In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
data = pd.read_excel('/content/gdrive/My Drive/NLP-Inaequo/DataPoints.xlsx', index_col=None)

In [ ]:
nltk.download('stopwords')
    # removing special characters and stop words from the text
stop_words_l=stopwords.words('english')
data['cleaned_answers']=data.Answers.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
i = 0
while i < (len(data.index)-2):
  for j in range(3):
    data['Original'][i+j] = data['Original'][i]
  i = i+3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
data['cleaned_original']=data.Original.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
data['TF/IDF Score']= np.nan
data

,Original,Answers,Points,cleaned_answers,cleaned_original,TF/IDF Score
0,"Mark is at the park. His dog Max, is with him....","Mark is playing with his dog, Max, at the park...",85,mark playing dog max park flies kite runs ...,mark park dog max him mark flies kite plays...,NaN
1,"Mark is at the park. His dog Max, is with him....",Mark is in the park. He is playing with his do...,65,mark park playing dog also flying kite mark...,mark park dog max him mark flies kite plays...,NaN
2,"Mark is at the park. His dog Max, is with him....",Mark and his dog are playing under the sun at ...,70,mark dog playing sun park fly kite go see du...,mark park dog max him mark flies kite plays...,NaN
3,Sir Isaac Newton was seated in his garden on a...,It was summer when Newton saw an apple fall fr...,70,summer newton saw apple fall tree started thi...,sir isaac newton seated garden summer s evenin...,NaN
4,Sir Isaac Newton was seated in his garden on a...,Sir Isaac Newton was in his garden when an apl...,90,sir isaac newton garden aplle fell tree start...,sir isaac newton seated garden summer s evenin...,NaN
...,...,...,...,...,...,...
295,The virus can spread from an infected person’s...,You can be tainted by taking in the infection ...,80,tainted taking infection case close somebody c...,virus spread infected person s mouth nose smal...,NaN
296,The virus can spread from an infected person’s...,Coronavirus disease (COVID-19) is an infectiou...,10,coronavirus disease covid infectious dise...,virus spread infected person s mouth nose smal...,NaN
297,A circle is a shape consisting of all points i...,A circle is a shape consesting of all points i...,80,circle shape consesting points plain given dis...,circle shape consisting points plane given dis...,NaN
298,A circle is a shape consisting of all points i...,a round plane figure whose boundary (the circu...,90,round plane figure whose boundary the circumf...,circle shape consisting points plane given dis...,NaN


From medium https://towardsdatascience.com/how-to-rank-text-content-by-semantic-similarity-4d2419a84c32

In [ ]:
i = 0
while i < len(data.index)-2:
  original = data["cleaned_original"][i]
  ans = []
  for j in range(3):
    ans.append(data["cleaned_answers"][i+j])
  doc_vectors = TfidfVectorizer().fit_transform([original] + ans)
  cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
  document_scores = [item.item() for item in cosine_similarities[1:]]
  for k in range(3):
    data["TF/IDF Score"][i+k]= document_scores[k]
  i = i + 3
  data.head

In [ ]:
data

,Original,Answers,Points,cleaned_answers,cleaned_original,TF/IDF Score
0,"Mark is at the park. His dog Max, is with him....","Mark is playing with his dog, Max, at the park...",85,mark playing dog max park flies kite runs ...,mark park dog max him mark flies kite plays...,0.608046
1,"Mark is at the park. His dog Max, is with him....",Mark is in the park. He is playing with his do...,65,mark park playing dog also flying kite mark...,mark park dog max him mark flies kite plays...,0.302161
2,"Mark is at the park. His dog Max, is with him....",Mark and his dog are playing under the sun at ...,70,mark dog playing sun park fly kite go see du...,mark park dog max him mark flies kite plays...,0.405026
3,Sir Isaac Newton was seated in his garden on a...,It was summer when Newton saw an apple fall fr...,70,summer newton saw apple fall tree started thi...,sir isaac newton seated garden summer s evenin...,0.606609
4,Sir Isaac Newton was seated in his garden on a...,Sir Isaac Newton was in his garden when an apl...,90,sir isaac newton garden aplle fell tree start...,sir isaac newton seated garden summer s evenin...,0.519421
...,...,...,...,...,...,...
295,The virus can spread from an infected person’s...,You can be tainted by taking in the infection ...,80,tainted taking infection case close somebody c...,virus spread infected person s mouth nose smal...,0.044100
296,The virus can spread from an infected person’s...,Coronavirus disease (COVID-19) is an infectiou...,10,coronavirus disease covid infectious dise...,virus spread infected person s mouth nose smal...,0.045266
297,A circle is a shape consisting of all points i...,A circle is a shape consesting of all points i...,80,circle shape consesting points plain given dis...,circle shape consisting points plane given dis...,0.606599
298,A circle is a shape consisting of all points i...,a round plane figure whose boundary (the circu...,90,round plane figure whose boundary the circumf...,circle shape consisting points plane given dis...,0.143384


In [ ]:
data.to_excel('/content/gdrive/My Drive/NLP-Inaequo/TF_IDF_scores.xlsx') 